In [2]:
from pysal.lib import weights
from matplotlib import pyplot as plt

import pandas as pd
import geopandas as gpd
import folium 
import segregation as seg

import sys
import os

os.environ["USE_PYGEOS"] = "0"
import geopandas as gpd

sys.path.insert(0, "../src/helper")

from landprice_merger import (
    zensus_landprice_merger,
    landprice_neighborhood_merger,
    combine_landprice_with_geodata,
)
from csv_merger import combine_csvs
from principal_component_analysis import combine_PCA_datasets


In [3]:
cities = ["Berlin", "Bremen", "Dresden", "Frankfurt", "Köln"]

main_path = os.path.join(os.getcwd(), "..")

path_zensus = os.path.join(main_path, "res", "data", "DLR", "2 Zensus")
path_land = os.path.join(main_path, "res", "data", "DLR", "1 Land Prices")
path_neigh = os.path.join(main_path, "res", "data", "DLR", "3 Neighborhoods")


In [7]:
df_zensus = combine_csvs(str_path=path_zensus)

gdf_zensus = combine_PCA_datasets(
    df_zensus=df_zensus, str_city=cities[0], str_path=path_zensus
)


In [ ]:

single_var_1 = "j_1979_1995"
single_var_2 = "we_housing_cooperative"
total_pop_var = "households_total_units"


df_zensus_bremen[total_pop_var] = df_zensus_bremen[total_pop_var].astype(int) # do this in utils
df_zensus_bremen[single_var_1] = df_zensus_bremen[single_var_1].astype(int)
df_zensus_bremen[single_var_2] = df_zensus_bremen[single_var_2].astype(int)

In [ ]:

# A-spatial segregation index     
int1 = seg.singlegroup.Interaction(data = df_zensus_bremen, group_pop_var = single_var_1, total_pop_var = total_pop_var)
int2 = seg.singlegroup.Interaction(data = df_zensus_bremen, group_pop_var = single_var_2, total_pop_var = total_pop_var)

print("Interaction of owner category %s: %.2f and Interaction of owner category %s: %.2f" %(single_var_1, int1.statistic, single_var_2, int2.statistic))


In [ ]:
zensus_bremen_grid = gpd.read_file("D:/ifo_hack/ifoHack_DLR_Challenge_Data/2 Zensus/Zensus_Bremen_Grid_100m.gpkg")
idx_column = "Grid_Code"


wr = weights.contiguity.Queen.from_dataframe(zensus_bremen_grid, geom_col = "geometry", ids = idx_column )    
zensus_bremen_grid_copy = zensus_bremen_grid[~zensus_bremen_grid[idx_column].isin(wr.islands)].copy()

m = zensus_bremen_grid.explore(height=500, width=1000, color="gray", name="Zensus Grid Cells 100mx100m")
m = zensus_bremen_grid_copy.explore(m=m, color="blue", name="Zensus Grid Cells (filtered)")

folium.LayerControl().add_to(m)
plt.show()

In [ ]:
dint1 = seg.singlegroup.DistanceDecayInteraction(data = zensus_bremen_grid_copy, group_pop_var = single_var_1, total_pop_var = total_pop_var)
dint2 = seg.singlegroup.DistanceDecayInteraction(data = zensus_bremen_grid_copy, group_pop_var = single_var_2, total_pop_var = total_pop_var)
    
print("Interaction (spatial) of age category %s: %.2f and Interaction (spatial) of age category %s: %.2f" %(single_var_1, dint1.statistic, 
                                                                                                            single_var_2, dint2.statistic))

zensus_bremen_grid_copy[single_var_1 + "_perc"] = [x * 100 / y if y != 0 else 0 for (x,y) in zip(zensus_bremen_grid_copy[single_var_1], zensus_bremen_grid_copy[total_pop_var])]
 
m = zensus_bremen_grid_copy.explore(height=500, width=1000, name="Seniors > 65yo",
                             column = single_var_1 + "_perc", scheme = "EqualInterval", cmap = "inferno", legend = True)


folium.LayerControl().add_to(m)
m